In [70]:
import numpy as np, pandas as pd
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
pd.set_option("display.max_rows", 50)

# data import and cleaning

In [139]:
econ

,DATE,formatted_date,UNRATE,INDPRO,FEDFUNDS,CPI,AMTMNO,RSXFS,home_sales,construction,monthly_gdp
0,2019-12-01,20191231,3.6,101.8840,1.55,258.62,508780,458.92B,103.4,1327663,21.74T
1,2020-01-01,20200101,3.5,101.3768,1.55,259.04,459272,456.11B,108.7,1369223,21.79T
2,2020-02-01,20200201,3.5,101.6330,1.58,259.25,460927,457.90B,110.8,1366697,21.92T
3,2020-03-01,20200301,4.4,97.6672,0.65,258.12,416951,432.39B,90.3,1360512,20.74T
4,2020-04-01,20200401,14.7,84.5979,0.05,256.09,356382,375.92B,70.0,1346166,18.51T
5,2020-05-01,20200501,13.2,85.9732,0.05,255.87,377550,442.04B,101.4,1356409,19.37T
6,2020-06-01,20200601,11.0,91.5625,0.08,256.99,408711,472.25B,115.0,1355189,20.56T
7,2020-07-01,20200701,10.2,95.0144,0.09,258.28,431781,481.17B,121.0,1364565,20.85T
8,2020-08-01,20200801,8.4,95.8881,0.10,259.41,436444,482.39B,130.3,1412823,21.34T
9,2020-09-01,20200901,7.9,95.8444,0.09,260.03,441723,492.70B,126.9,1414027,21.65T


In [141]:
econ = pd.read_csv("../data/econ_factors.csv")
# econ = econ[["formatted_date","UNRATE",	"INDPRO","FEDFUNDS", "CPI",	"monthly_gdp"]]
econ.drop("DATE",axis=1, inplace = True)

In [394]:
# covid

def clean_covid():
    covid = pd.read_csv("../data/covid.csv")
    covid = covid[['new_cases','formatted_date']]
    covid = covid.query('(formatted_date >= 20200101) & (formatted_date <= 20211231)')

    covid = covid.set_index('formatted_date')
    covid.index = pd.to_datetime(covid.index, format='%Y%m%d')
    return covid

# snp
def clean_snp():
    snp = pd.read_csv("../data/snp.csv")
    snp = snp[["formatted_date","SnP_Close","SnP_Volume","VIX"]]

    snp["snp_close_pct_change"] = snp["SnP_Close"].pct_change()
    snp["snp_volume_pct_change"] = snp["SnP_Volume"].pct_change()
    snp["vix_pct_change"] = snp["VIX"].pct_change()
    snp.fillna(0, inplace=True)

    snp = snp[['formatted_date','snp_close_pct_change','snp_volume_pct_change','vix_pct_change']]

    snp = snp.set_index('formatted_date')
    snp.index = pd.to_datetime(snp.index, format='%Y%m%d')
    return snp


# econ
def clean_econ():
    econ = pd.read_csv("../data/econ_factors.csv")
    # econ = econ[["formatted_date","UNRATE",	"INDPRO","FEDFUNDS", "CPI",	"monthly_gdp"]]

    ###
    econ.drop("DATE",axis=1, inplace=True)
    econ["RSXFS"] = econ["RSXFS"].apply(lambda x: x.replace("B", ""))
    econ["RSXFS"] = econ["RSXFS"].astype(float)
    ###

    econ["monthly_gdp"] = econ["monthly_gdp"].apply(lambda x: x.replace("T", ""))
    econ["monthly_gdp"] = econ["monthly_gdp"].astype(float)



    # monthly data interpolation
    econ = econ.set_index('formatted_date')
    econ.index = pd.to_datetime(econ.index, format='%Y%m%d')
    econ_daily = econ.resample('D').interpolate()
    return econ_daily

# merging

feature_df = pd.merge(clean_covid(), clean_snp(), on='formatted_date', how='left')
feature_df = pd.merge(feature_df, clean_econ(), on='formatted_date', how='left')
feature_df.fillna(method='ffill', inplace=True)

# scale every column using min-max scaler
scaler = MinMaxScaler()
feature_df[feature_df.columns] = scaler.fit_transform(feature_df[feature_df.columns])

feature_df

/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.ax is deprecated and will be removed in a future version. Use Resampler.ax instead
  econ_daily = econ.resample('D').interpolate()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.indexer is deprecated and will be removed in a future version. Use Resampler.indexer instead.
  econ_daily = econ.resample('D').interpolate()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.obj is deprecated and will be removed in a future version. Use GroupBy.indexer instead.
  econ_daily = econ.resample('D').interpolate()


,new_cases,snp_close_pct_change,snp_volume_pct_change,vix_pct_change,UNRATE,INDPRO,FEDFUNDS,CPI,AMTMNO,RSXFS,home_sales,construction,monthly_gdp
formatted_date,,,,,,,,,,,,,
2020-01-01,0.000000,0.560872,0.283023,0.274932,0.000000,0.984960,0.980392,0.118839,0.628259,0.404774,0.641791,0.078506,0.545757
2020-01-02,0.000000,0.600086,0.393199,0.163111,0.000000,0.985446,0.981025,0.119093,0.628585,0.405066,0.642914,0.078228,0.546455
2020-01-03,0.000000,0.527831,0.287063,0.421139,0.000000,0.985931,0.981657,0.119346,0.628911,0.405357,0.644038,0.077951,0.547153
2020-01-04,0.000000,0.527831,0.287063,0.421139,0.000000,0.986416,0.982290,0.119600,0.629237,0.405649,0.645161,0.077673,0.547850
2020-01-05,0.000000,0.527831,0.287063,0.421139,0.000000,0.986901,0.982922,0.119854,0.629563,0.405940,0.646285,0.077396,0.548548
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.244195,0.625641,0.255370,0.256594,0.043203,0.962880,0.019608,0.991728,0.990742,0.991123,0.661585,0.742785,1.000000
2021-12-28,0.445456,0.556144,0.270318,0.265618,0.043491,0.963139,0.019608,0.993796,0.993056,0.993342,0.657626,0.732893,1.000000
2021-12-29,0.979309,0.567433,0.336245,0.235366,0.043779,0.963397,0.019608,0.995864,0.995371,0.995561,0.653667,0.723000,1.000000


In [245]:
test_df

,new_cases,snp_close_pct_change,vix_pct_change,new_cases_pct_change
formatted_date,,,,
2020-01-08,0.0,0.004902,-0.024656,0.000000
2020-01-09,0.0,0.006655,-0.067658,0.000000
2020-01-10,0.0,-0.002855,0.001595,0.000000
2020-01-11,0.0,-0.002855,0.001595,0.000000
2020-01-12,0.0,-0.002855,0.001595,0.000000
...,...,...,...,...
2021-12-27,87584.0,0.013839,-0.015590,-0.441297
2021-12-28,102213.0,-0.001010,-0.007919,0.167028
2021-12-29,266159.0,0.001402,-0.033637,1.603964


# Model Dev

## Iter 1

In [205]:
# Select the input variables: new_cases, snp_volume_pct_change, vix_pct_change, UNRATE, INDPRO, FEDFUNDS, CPI, monthly_gdp
X = feature_df[['new_cases', 'snp_volume_pct_change', 'vix_pct_change', 'UNRATE', 'INDPRO', 'FEDFUNDS', 'CPI', 'monthly_gdp']]

# Add a constant term to the predictors (required for statsmodels)
X = sm.add_constant(X)

# Select the output variable: snp_close_pct_change
y = feature_df[['snp_close_pct_change']]

# Split the dataset into training and testing sets, maintaining the time order of the data
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create a linear regression model
model = sm.OLS(y_train, X_train)

# Fit the model to the training data
results = model.fit()

# Print the summary
print(results.summary())

# Make predictions using the test set
y_pred = results.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.2f}")

# Calculate the R-squared score
r2 = r2_score(y_test, y_pred)
print(f"R-squared score: {r2:.2f}")

# Conclusion:
# Summarize the findings and discuss any insights gained from the analysis. Address any limitations of the study and suggest possible improvements or future research directions.

                             OLS Regression Results                             
Dep. Variable:     snp_close_pct_change   R-squared:                       0.804
Model:                              OLS   Adj. R-squared:                  0.647
Method:                   Least Squares   F-statistic:                     5.122
Date:                  Sun, 23 Apr 2023   Prob (F-statistic):            0.00955
Time:                          02:39:07   Log-Likelihood:                 14.903
No. Observations:                    19   AIC:                            -11.81
Df Residuals:                        10   BIC:                            -3.307
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const     

/Users/edward.w/opt/anaconda3/envs/COMP3115/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


## Iter 2 (gp by month)

In [203]:
# covid

covid = pd.read_csv("../data/covid.csv")
covid = covid[['new_cases','formatted_date']]
covid = covid.query('(formatted_date >= 20200101) & (formatted_date <= 20211231)')

covid = covid.set_index('formatted_date')
covid.index = pd.to_datetime(covid.index, format='%Y%m%d')
covid = covid.resample('M').sum()

# snp

snp = pd.read_csv("../data/snp.csv")
snp = snp[["formatted_date","SnP_Close","SnP_Volume","VIX"]]

snp["snp_close_pct_change"] = snp["SnP_Close"].pct_change()
snp["snp_volume_pct_change"] = snp["SnP_Volume"].pct_change()
snp["vix_pct_change"] = snp["VIX"].pct_change()
snp.fillna(0, inplace=True)

snp = snp[['formatted_date','snp_close_pct_change','snp_volume_pct_change','vix_pct_change']]

snp = snp.set_index('formatted_date')
snp.index = pd.to_datetime(snp.index, format='%Y%m%d')
snp = snp.resample('M').mean()

# econ

econ = pd.read_csv("../data/econ_factors.csv")
# econ = econ[["formatted_date","UNRATE",	"INDPRO","FEDFUNDS", "CPI",	"monthly_gdp"]]
econ.drop("DATE",axis=1, inplace=True)

econ["monthly_gdp"] = econ["monthly_gdp"].apply(lambda x: x.replace("T", ""))
econ["monthly_gdp"] = econ["monthly_gdp"].astype(float)
econ["RSXFS"] = econ["RSXFS"].apply(lambda x: x.replace("B", ""))
econ["RSXFS"] = econ["RSXFS"].astype(float)

econ = econ.set_index('formatted_date')
econ.index = pd.to_datetime(econ.index, format='%Y%m%d')
econ = econ.resample('M').last()
# Add lags to the economic variables
lag_months = 1  # You can change this value to experiment with different lag periods
econ_lagged = econ.shift(lag_months).dropna()

# Merging with the lagged econ dataframe
feature_df = pd.merge(covid, snp, on='formatted_date', how='left')
feature_df = pd.merge(feature_df, econ_lagged, on='formatted_date', how='left')
feature_df.fillna(method='ffill', inplace=True)

# Scale every column using min-max scaler
scaler = MinMaxScaler()
feature_df[feature_df.columns] = scaler.fit_transform(feature_df[feature_df.columns])

feature_df


/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/4251669730.py:9: FutureWarning: TimeGrouper.ax is deprecated and will be removed in a future version. Use Resampler.ax instead
  covid = covid.resample('M').sum()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/4251669730.py:9: FutureWarning: TimeGrouper.indexer is deprecated and will be removed in a future version. Use Resampler.indexer instead.
  covid = covid.resample('M').sum()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/4251669730.py:9: FutureWarning: TimeGrouper.obj is deprecated and will be removed in a future version. Use GroupBy.indexer instead.
  covid = covid.resample('M').sum()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/4251669730.py:25: FutureWarning: TimeGrouper.ax is deprecated and will be removed in a future version. Use Resampler.ax instead
  snp = snp.resample('M').mean()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/425166973

,new_cases,snp_close_pct_change,snp_volume_pct_change,vix_pct_change,UNRATE,INDPRO,FEDFUNDS,CPI,AMTMNO,RSXFS,home_sales,construction,monthly_gdp
formatted_date,,,,,,,,,,,,,
2020-01-31,0.000000,0.422827,0.511543,0.594433,0.008929,1.000000,0.980392,0.120403,1.000000,0.434010,0.553897,0.000000,0.556897
2020-02-29,0.000008,0.000000,0.717696,1.000000,0.000000,0.970659,0.980392,0.138792,0.675140,0.419316,0.641791,0.139361,0.565517
2020-03-31,0.027534,0.009997,0.199111,0.712747,0.000000,0.985480,1.000000,0.147986,0.686000,0.428676,0.676617,0.130890,0.587931
2020-04-30,0.136372,1.000000,0.331523,0.128288,0.080357,0.756058,0.392157,0.098511,0.397440,0.295283,0.336650,0.110151,0.384483
2020-05-31,0.120136,0.647625,0.464818,0.261593,1.000000,0.000000,0.000000,0.009632,0.000000,0.000000,0.000000,0.062045,0.000000
2020-06-30,0.130648,0.522060,0.506468,0.496615,0.866071,0.079561,0.000000,0.000000,0.138899,0.345744,0.520730,0.096392,0.148276
2020-07-31,0.299775,0.662700,0.303751,0.262636,0.669643,0.402902,0.019608,0.049037,0.343371,0.503713,0.746269,0.092301,0.353448
2020-08-31,0.242549,0.735705,0.185269,0.428241,0.598214,0.602594,0.026144,0.105517,0.494751,0.550356,0.845771,0.123741,0.403448
2020-09-30,0.191187,0.257471,0.469984,0.397995,0.437500,0.653137,0.032680,0.154991,0.525348,0.556735,1.000000,0.285562,0.487931


In [204]:
# Select the input variables: new_cases, snp_volume_pct_change, vix_pct_change, UNRATE, INDPRO, FEDFUNDS, CPI, monthly_gdp
X = feature_df[['snp_close_pct_change', 'UNRATE', 'INDPRO', 'FEDFUNDS', 'CPI', 'monthly_gdp']]

# Add a constant term to the predictors (required for statsmodels)
X = sm.add_constant(X)

# Select the output variable: snp_close_pct_change
y = feature_df[['vix_pct_change']]

# Split the dataset into training and testing sets, maintaining the time order of the data
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create a linear regression model
model = sm.OLS(y_train, X_train)

# Fit the model to the training data
results = model.fit()

# Print the summary
print(results.summary())

# Make predictions using the test set
y_pred = results.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.2f}")

# Calculate the R-squared score
r2 = r2_score(y_test, y_pred)
print(f"R-squared score: {r2:.2f}")

# Conclusion:
# Summarize the findings and discuss any insights gained from the analysis. Address any limitations of the study and suggest possible improvements or future research directions.

                            OLS Regression Results                            
Dep. Variable:         vix_pct_change   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     7.524
Date:                Sun, 23 Apr 2023   Prob (F-statistic):            0.00162
Time:                        02:38:42   Log-Likelihood:                 15.925
No. Observations:                  19   AIC:                            -17.85
Df Residuals:                      12   BIC:                            -11.24
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.5034 

/Users/edward.w/opt/anaconda3/envs/COMP3115/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# iter 3

In [420]:
lagcov = clean_covid().shift(0).dropna()
temp_snp = clean_snp()[["snp_close_pct_change","vix_pct_change"]]

iter3_df = pd.merge(lagcov,temp_snp, how="left", on="formatted_date")
iter3_df.fillna(method="ffill",inplace = True)
iter3_df["new_cases_pct_change"] = iter3_df["new_cases"].pct_change()
iter3_df["new_cases_pct_change"].fillna(0, inplace=True)


iter3_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values using your preferred method, for example, forward fill
iter3_df.fillna(method="ffill", inplace=True)

# If there are still any NaN values remaining, you can fill them with 0 or any other value
iter3_df.fillna(0, inplace=True)




# Now you can apply the MinMaxScaler
scaler = MinMaxScaler()
iter3_df[iter3_df.columns] = scaler.fit_transform(iter3_df[iter3_df.columns])
iter3_df

wti = pd.read_csv("../data/WTI.csv")
tbill = pd.read_csv("../data/3MTbill.csv")
saving = pd.read_csv("../data/PSAVERT.csv")

# tbill
tbill = tbill[(tbill["DATE"]>="2019-12-31")&(tbill["DATE"]<="2022-01-01")]
tbill.replace(".", np.nan, inplace=True)
tbill["DTB3"].fillna(method="ffill",inplace=True)
tbill['DATE'] = pd.to_datetime(tbill['DATE'], format='%Y-%m-%d')

# saving
saving = saving[(saving["DATE"]>="2020-01-01")&(saving["DATE"]<="2022-01-01")]

# wti
wti['Day'] = pd.to_datetime(wti['Day'], format='%m/%d/%Y')
wti['DATE'] = wti['Day'].dt.strftime('%Y-%m-%d')
wti['DATE'] = pd.to_datetime(wti['DATE'], format='%Y-%m-%d')
wti = wti[(wti["DATE"]>="2019-12-31")&(wti["DATE"]<="2022-01-01")]
wti.drop("Day",axis=1,inplace= True)
wti.rename({"Price":"wti"},axis=1,inplace=True)

added_feature_iter3_df = pd.merge(covid,tbill,left_index=True, right_on="DATE",how="outer")
added_feature_iter3_df['DATE'] = pd.to_datetime(added_feature_iter3_df['DATE'], format='%Y-%m-%d')
added_feature_iter3_df = pd.merge(added_feature_iter3_df, wti,left_on="DATE",right_on="DATE",how="outer")
added_feature_iter3_df.sort_values(by="DATE",inplace=True)
added_feature_iter3_df.fillna(method="ffill",inplace=True)
added_feature_iter3_df = added_feature_iter3_df.iloc[1:,1:]
added_feature_iter3_df.set_index("DATE",inplace=True)
added_feature_iter3_df[added_feature_iter3_df.columns] = scaler.fit_transform(added_feature_iter3_df[added_feature_iter3_df.columns])

iter3_final_df = pd.merge(added_feature_iter3_df, iter3_df, left_index=True, right_index=True)
iter3_final_df = pd.merge(iter3_final_df,clean_econ(),left_index=True,right_index=True)


/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.ax is deprecated and will be removed in a future version. Use Resampler.ax instead
  econ_daily = econ.resample('D').interpolate()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.indexer is deprecated and will be removed in a future version. Use Resampler.indexer instead.
  econ_daily = econ.resample('D').interpolate()
/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/3588474979.py:48: FutureWarning: TimeGrouper.obj is deprecated and will be removed in a future version. Use GroupBy.indexer instead.
  econ_daily = econ.resample('D').interpolate()


# Linear Regression

In [421]:
iter3_final_df

,DTB3,wti,new_cases,snp_close_pct_change,vix_pct_change,new_cases_pct_change,UNRATE,INDPRO,FEDFUNDS,CPI,AMTMNO,RSXFS,home_sales,construction,monthly_gdp
2020-01-01,0.975155,0.800196,0.000000,0.560872,0.274932,0.123331,3.500000,101.376800,1.550000,259.040000,459272.000000,456.110000,108.700000,1.369223e+06,21.790000
2020-01-02,0.968944,0.800440,0.000000,0.600086,0.163111,0.123331,3.500000,101.385065,1.550968,259.046774,459325.387097,456.167742,108.767742,1.369142e+06,21.794194
2020-01-03,0.956522,0.815365,0.000000,0.527831,0.421139,0.123331,3.500000,101.393329,1.551935,259.053548,459378.774194,456.225484,108.835484,1.369060e+06,21.798387
2020-01-04,0.956522,0.815365,0.000000,0.527831,0.421139,0.123331,3.500000,101.401594,1.552903,259.060323,459432.161290,456.283226,108.903226,1.368979e+06,21.802581
2020-01-05,0.956522,0.815365,0.000000,0.527831,0.421139,0.123331,3.500000,101.409858,1.553871,259.067097,459485.548387,456.340968,108.970968,1.368897e+06,21.806774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.068323,0.917224,0.244195,0.625641,0.256594,0.059162,3.983871,101.000652,0.080000,282.324194,518635.677419,572.271613,109.893548,1.564321e+06,24.520000
2021-12-28,0.068323,0.921465,0.445456,0.556144,0.265618,0.224979,3.987097,101.005061,0.080000,282.379355,519014.741935,572.711290,109.654839,1.561415e+06,24.520000
2021-12-29,0.062112,0.926113,0.979309,0.567433,0.235366,0.271137,3.990323,101.009471,0.080000,282.434516,519393.806452,573.150968,109.416129,1.558510e+06,24.520000
2021-12-30,0.062112,0.928152,0.821224,0.546880,0.301303,0.103423,3.993548,101.013881,0.080000,282.489677,519772.870968,573.590645,109.177419,1.555604e+06,24.520000


In [425]:
# feature selection

import statsmodels.api as sm

def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Added {:30} with p-value {:.6}'.format(best_feature, best_pval))
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Dropped {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# Assuming 'snp_close_pct_change' is the dependent variable
X = iter3_final_df.drop(columns=['vix_pct_change'])
y = iter3_final_df['vix_pct_change']

selected_features = stepwise_selection(X, y)

print('\nSelected features:')
print(selected_features)


Added snp_close_pct_change           with p-value 8.06463e-94
Added DTB3                           with p-value 0.00218644

Selected features:
['snp_close_pct_change', 'DTB3']


In [427]:
xcol = ['vix_pct_change','new_cases_pct_change','DTB3']
ycol = ['snp_close_pct_change']
running_regression(iter3_final_df,xcol,ycol)

                             OLS Regression Results                             
Dep. Variable:     snp_close_pct_change   R-squared:                       0.459
Model:                              OLS   Adj. R-squared:                  0.456
Method:                   Least Squares   F-statistic:                     164.0
Date:                  Sun, 23 Apr 2023   Prob (F-statistic):           5.73e-77
Time:                          18:00:07   Log-Likelihood:                 826.67
No. Observations:                   584   AIC:                            -1645.
Df Residuals:                       580   BIC:                            -1628.
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const       

In [426]:
xcol = ['snp_close_pct_change','new_cases_pct_change','DTB3']
ycol = ['vix_pct_change']
running_regression(iter3_final_df,xcol,ycol)

                            OLS Regression Results                            
Dep. Variable:         vix_pct_change   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     173.5
Date:                Sun, 23 Apr 2023   Prob (F-statistic):           2.75e-80
Time:                        16:12:32   Log-Likelihood:                 671.48
No. Observations:                 584   AIC:                            -1335.
Df Residuals:                     580   BIC:                            -1317.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7743 

In [369]:
def running_regression(df, xcol, ycol):
    # Select the input variables: new_cases, snp_volume_pct_change, vix_pct_change, UNRATE, INDPRO, FEDFUNDS, CPI, monthly_gdp
    X = df[xcol]

    # Add a constant term to the predictors (required for statsmodels)
    X = sm.add_constant(X)

    # Select the output variable: snp_close_pct_change
    y = df[ycol]

    # Split the dataset into training and testing sets, maintaining the time order of the data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Create a linear regression model
    model = sm.OLS(y_train, X_train)

    # Fit the model to the training data
    results = model.fit()

    # Print the summary
    print(results.summary())

    # Make predictions using the test set
    y_pred = results.predict(X_test)

    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean squared error: {mse:.2f}")

    # Calculate the R-squared score
    r2 = r2_score(y_test, y_pred)
    print(f"R-squared score: {r2:.2f}")

# Conclusion:
# Summarize the findings and discuss any insights gained from the analysis. Address any limitations of the study and suggest possible improvements or future research directions.

## random forest regression (not working)

In [377]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Prepare the data
x_cols = ['vix_pct_change','new_cases_pct_change', 'DTB3', 'wti']
y_cols = ['snp_close_pct_change']

# Split the data into training and testing sets
X = iter3_final_df[x_cols]
y = iter3_final_df[y_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate and display the metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean squared error: {:.4f}".format(mse))
print("R-squared score: {:.4f}".format(r2))


Mean squared error: 0.0032
R-squared score: 0.4673


/var/folders/h3/73y674_56hs86stnly2g7s740000gn/T/ipykernel_20460/1454069441.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_regressor.fit(X_train, y_train)


In [378]:
# Get feature importances
importances = rf_regressor.feature_importances_

# Map importances to the column names
feature_importances = pd.Series(importances, index=x_cols)

# Display the importances
print("Feature importances:")
print(feature_importances)


Feature importances:
vix_pct_change          0.799653
new_cases_pct_change    0.200347
dtype: float64
